# 3D Image Classification from CT Scans

**Author:** [Hasib Zunair](https://twitter.com/hasibzunair)<br>
**Date created:** 2020/09/23<br>
**Last modified:** 2020/09/23<br>
**Description:** Train a 3D convolutional neural network to predict presence of pneumonia.

## Introduction

This example will show the steps needed to build a 3D convolutional neural network (CNN)
to predict the presence of viral pneumonia in computer tomography (CT) scans. 2D CNNs are
commonly used to process RGB images (3 channels). A 3D CNN is simply the 3D
equivalent: it takes as input a 3D volume or a sequence of 2D frames (e.g. slices in a CT scan),
3D CNNs are a powerful model for learning representations for volumetric data.

## References

- [A survey on Deep Learning Advances on Different 3D DataRepresentations](https://arxiv.org/pdf/1808.01462.pdf)
- [VoxNet: A 3D Convolutional Neural Network for Real-Time Object Recognition](https://www.ri.cmu.edu/pub_files/2015/9/voxnet_maturana_scherer_iros15.pdf)
- [FusionNet: 3D Object Classification Using MultipleData Representations](http://3ddl.cs.princeton.edu/2016/papers/Hegde_Zadeh.pdf)
- [Uniformizing Techniques to Process CT scans with 3D CNNs for Tuberculosis Prediction](https://arxiv.org/abs/2007.13224)

## Setup

In [12]:
!pip install pip --upgrade -q
!pip install nibabel scipy matplotlib tensorflow -q

In [13]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import logging
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
logging.basicConfig(level=logging.DEBUG)

In [15]:
#
# Load model from storage.
#
import requests
url = "https://koz.s3.amazonaws.com/models/3d_image_classification.h5"
model_file = '3d_image_classification.h5'

filename = os.path.join(os.getcwd(), model_file)
keras.utils.get_file(filename, url)

model = keras.models.load_model(filename)

In [16]:
#
# Load volume data from storage.
#
url = "https://koz.s3.amazonaws.com/data/ct-data.zip"
filename = os.path.join(os.getcwd(), "ct-data.zip")
keras.utils.get_file(filename, url)

# Unzip data in the newly created directory.
with zipfile.ZipFile("ct-data.zip", "r") as z_fp:
    z_fp.extractall("./")

In [17]:
import nibabel as nib
from scipy import ndimage

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


## Make predictions on a single CT scan

In [27]:
import requests
#
# payload format
# payload = {"data": {"ndarray": X.tolist()} }
#

for i in range(5):
    filename = f'./data/volume{i}.nii.gz'
    v = read_nifti_file(filename)
    #
    # Prediction via REST.
    #
    url = 'http://mymodel-mygroup-ml-mon.2886795275-80-kota02.environments.katacoda.com/api/v1.0/predictions'
    print(f'Serializing and predicting volume {i}')
    payload = {"data": {"ndarray": v.tolist()} }
    r = requests.post(url, json = payload)
    logging.debug(f'response: {r}')
    j = r.json()['data']['ndarray']
    print(f'Volume {i} prediction: {j}')
    
#     class_names = ["normal", "abnormal"]
#     print(f'Volume : Prediction for pneumonia is {(prediction > 0.5)}, probability = {100 * prediction[0]:.1f}%, label = {class_names[volume]==1}')


Serializing and predicting volume 0
Volume 0 prediction: [0.8098430633544922]
Serializing and predicting volume 1
Volume 1 prediction: [0.7982639670372009]
Serializing and predicting volume 2
Volume 2 prediction: [0.6138761043548584]
Serializing and predicting volume 3
Volume 3 prediction: [0.4892294108867645]
Serializing and predicting volume 4
Volume 4 prediction: [0.9624292850494385]


In [23]:
r

<Response [200]>